# 🎵 DEMO: Phát Hiện Ngáy từ Audio File

Script đơn giản để test model với file audio

## 📝 Hướng dẫn sử dụng:
1. **Thay đổi đường dẫn file audio** ở cell bên dưới (dòng `AUDIO_FILE`)
2. **Chạy tất cả các cell** (Cell → Run All)
3. **Xem kết quả** ở cuối notebook

## ⚙️ Yêu cầu:
- File model: `Model_LeNet5_PhatHienNgay_100x64.h5` phải có trong cùng thư mục
- File audio: WAV, MP3, M4A, FLAC, OGG, AAC


In [1]:
# ============================================
# ⚙️ CẤU HÌNH - CHỈ CẦN THAY ĐƯỜNG DẪN FILE Ở ĐÂY
# ============================================

# 👇 THAY ĐƯỜNG DẪN FILE AUDIO CỦA BẠN VÀO ĐÂY 👇
AUDIO_FILE = "/Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/TIẾNG NGÁY NGỦ.mp3"  # Ví dụ: "audio/test.wav" hoặc "/Users/name/audio.mp3"

MODEL_FILE = "Model_LeNet5_PhatHienNgay_100x64.h5"

# Tên các lớp
CLASS_NAMES = ['OSA Snore', 'Normal Snore', 'Others']

print("✅ Đã cấu hình!")
print(f"📂 File audio: {AUDIO_FILE}")
print(f"🤖 File model: {MODEL_FILE}")


✅ Đã cấu hình!
📂 File audio: /Users/thanhhuongtran/Documents/lab-proj/mentor/D2T/TIẾNG NGÁY NGỦ.mp3
🤖 File model: Model_LeNet5_PhatHienNgay_100x64.h5


In [2]:
# Import các thư viện cần thiết
import numpy as np
import librosa
import pywt
from scipy import ndimage
from tensorflow.keras.models import load_model
import os

print("✅ Đã import tất cả thư viện!")


ModuleNotFoundError: No module named 'pywt'

In [ ]:
# Kiểm tra file có tồn tại không
if not os.path.exists(AUDIO_FILE):
    print(f"❌ LỖI: Không tìm thấy file audio!")
    print(f"   Đường dẫn: {AUDIO_FILE}")
    print(f"\n💡 Hãy thay đổi AUDIO_FILE ở cell trên!")
else:
    print(f"✅ Đã tìm thấy file audio: {AUDIO_FILE}")

if not os.path.exists(MODEL_FILE):
    print(f"❌ LỖI: Không tìm thấy file model!")
    print(f"   Đường dẫn: {MODEL_FILE}")
else:
    print(f"✅ Đã tìm thấy file model: {MODEL_FILE}")


In [ ]:
# Load model
print(f"📦 Đang load model từ: {MODEL_FILE}...")
model = load_model(MODEL_FILE)
print("✅ Đã load model thành công!")


In [ ]:
# Hàm chuyển đổi audio thành DWT spectrogram
def audio_to_dwt_spectrogram(audio_path, target_shape=(100, 64)):
    """
    Chuyển đổi audio file thành DWT spectrogram với kích thước 100x64
    """
    print(f"📂 Đang đọc file audio: {audio_path}")
    
    # Load audio file
    y, sr = librosa.load(audio_path, sr=None, mono=True)
    print(f"   ✅ Đã load audio: {len(y)/sr:.2f} giây, sample rate: {sr} Hz")
    
    # Nếu audio quá ngắn, pad thêm
    min_length = sr * 1  # Tối thiểu 1 giây
    if len(y) < min_length:
        y = np.pad(y, (0, min_length - len(y)), mode='constant')
        print(f"   ⚠️  Audio quá ngắn, đã pad thêm")
    
    # Chuyển đổi sang DWT spectrogram
    print("   🔄 Đang chuyển đổi sang DWT spectrogram...")
    coeffs = pywt.wavedec(y, 'db4', level=6)  # Daubechies 4, level 6
    
    # Kết hợp các hệ số DWT
    dwt_matrix = []
    for coeff in coeffs:
        coeff_resized = librosa.util.fix_length(coeff, size=target_shape[1])
        dwt_matrix.append(coeff_resized)
    
    # Kết hợp tất cả các hệ số
    dwt_combined = np.vstack(dwt_matrix)
    
    # Resize về kích thước mong muốn
    dwt_resized = ndimage.zoom(dwt_combined, 
                               (target_shape[0] / dwt_combined.shape[0], 
                                target_shape[1] / dwt_combined.shape[1]),
                               order=1)
    
    # Normalize về [0, 1]
    if dwt_resized.max() > dwt_resized.min():
        dwt_resized = (dwt_resized - dwt_resized.min()) / (dwt_resized.max() - dwt_resized.min())
    
    print(f"   ✅ Đã tạo spectrogram: {dwt_resized.shape}")
    return dwt_resized.astype(np.float32)

print("✅ Đã định nghĩa hàm xử lý audio!")


In [ ]:
# Chuyển đổi audio thành spectrogram
print("="*50)
print("🔍 ĐANG PHÂN TÍCH AUDIO...")
print("="*50)

spectrogram = audio_to_dwt_spectrogram(AUDIO_FILE)

# Reshape để phù hợp với input của model (1, 100, 64, 1)
spectrogram_reshaped = spectrogram.reshape(1, 100, 64, 1)

print(f"\n✅ Đã chuẩn bị dữ liệu cho model!")


In [ ]:
# Chạy model predict
print("\n🤖 Đang chạy model predict...")
predictions = model.predict(spectrogram_reshaped, verbose=0)

# Lấy kết quả
class_idx = np.argmax(predictions[0])
confidence = float(predictions[0][class_idx])
class_name = CLASS_NAMES[class_idx]

print("✅ Đã hoàn tất predict!")


In [ ]:
# Hiển thị kết quả
print("\n" + "="*50)
print("📊 KẾT QUẢ DỰ ĐOÁN")
print("="*50)
print(f"\n🎯 Lớp dự đoán: {class_name}")
print(f"📈 Độ tin cậy: {confidence*100:.2f}%")

print(f"\n📋 Xác suất cho tất cả các lớp:")
print("-" * 50)
for i, (name, prob) in enumerate(zip(CLASS_NAMES, predictions[0])):
    bar_length = int(prob * 30)  # Bar dài 30 ký tự
    bar = "█" * bar_length + "░" * (30 - bar_length)
    marker = "👉" if i == class_idx else "  "
    print(f"{marker} {name:15s}: {prob*100:6.2f}% {bar}")

print("="*50)


In [ ]:
# Hiển thị kết quả dạng dictionary (để dễ sử dụng)
results = {
    'predicted_class': class_name,
    'confidence': round(confidence * 100, 2),
    'all_probabilities': {
        CLASS_NAMES[0]: round(float(predictions[0][0]) * 100, 2),
        CLASS_NAMES[1]: round(float(predictions[0][1]) * 100, 2),
        CLASS_NAMES[2]: round(float(predictions[0][2]) * 100, 2)
    }
}

print("\n📦 Kết quả dạng dictionary:")
results
